In [39]:
import pandas as pd
import numpy as np
from collections import Counter

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch
import re
import string
import codecs
import unidecode

from nltk.corpus import stopwords
import nltk
from string import punctuation
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

import warnings
warnings.filterwarnings("ignore")

In [40]:
train_data = pd.read_csv('data/train.csv',encoding='utf-8')
test_data = pd.read_csv('data/test.csv',encoding='utf-8')

In [41]:
train_data.dropna(inplace=True)
train_data.isna().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [42]:
from bs4 import BeautifulSoup
def remove_html_encodings(x):
    example1 = BeautifulSoup(x, 'lxml')
    return example1.get_text()

In [43]:
train_data["tweet"] = train_data["tweet"].apply(lambda x: remove_html_encodings(x))

In [44]:
test_data['tweet'] = test_data['tweet'].apply(lambda x: remove_html_encodings(x))

In [45]:
train_data = train_data[train_data['sentiment'] != 3]

In [46]:
def clean_text(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
#     print(text)
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub("’", "'", decoded)
    #text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text) #""" removes any words in square brackets"""
    #text = re.sub('{link}', '', text) #"""removes {link} from text """
    #text = re.sub('@mention','',text) #removes user handles
    #text = re.sub('rt','',text) #removes RT as string
    text = re.sub('<.*?>+', '', text) #"""removes any words in <___> """
    text = re.sub('[^\s]*sxsw[^\s]*','',text) #removes all strings with sxsw
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #"""removes punctuations """
    
    text = re.sub('\n', '', text) #removes line breaks
    text = re.sub('\w*\d\w*', '', text) #removes words with digits
    text = re.sub('[0-9a-zA-Z]*[^\s0-9a-zA-Z]+[0-9a-zA-Z]*','',text)
    text = re.sub(r'(.)\1+', r'\1\1', text) #spell_corrected
    text = text.replace('\r','')
    return text

train_data['ctext'] = train_data['tweet'].apply(lambda x:clean_text(x))
test_data['ctext'] = test_data['tweet'].apply(lambda x:clean_text(x))


In [47]:
# Removing Stopwords
stopwords = set(STOPWORDS)
more_stopwords = {'amp', "quot","via","will"}
stopwords = stopwords.union(more_stopwords)

included_stop_words = ['nobody','not','n‘t','less',"n't",'no','never','cannot','full','none','first','least','top','nothing','against','nobody','everyone']

new_stop_words = set(i for i in stopwords if i not in included_stop_words)
#print(new_stop_words)

train_data["ctext"] = train_data["ctext"].apply(lambda x : " ".join([t for t in x.split() if t not in new_stop_words]))
test_data["ctext"] = test_data["ctext"].apply(lambda x : " ".join([t for t in x.split() if t not in new_stop_words]))

In [48]:
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
train_data['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in train_data['ctext']] 
print(train_data['tokenized_text'].head())

0    [apple, defining, language, touch, different, ...
1    [learning, ab, google, doodles, doodles, light...
2    [one, inyourface, ex, stealing, show, yrs, rt,...
3    [iphone, app, pretty, awesome, didnt, crash, e...
4    [line, outside, apple, store, austin, waiting,...
Name: tokenized_text, dtype: object


In [49]:
test_data['tokenized_text'] = [simple_preprocess(line,deacc=True) for line in test_data['ctext']]
print(test_data['tokenized_text'].head())

0    [audience, prototyping, tools, use, pens, phot...
1    [send, best, photos, videos, link, cnn, irepor...
2    [mention, heres, pic, winning, ipad, unsix, cc...
3    [google, marissa, mayer, mobile, phone, cursor...
4                [google, maps, even, cooler, thought]
Name: tokenized_text, dtype: object


In [50]:
# porter = PorterStemmer()
lemmatizer =  WordNetLemmatizer()
# stemming words
# stemmer = PorterStemmer()
corpus = []
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x:[lemmatizer.lemmatize(i) for i in x])
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x:' '.join(x))
#for i in train_data.tokenized_text:
 #   corpus.append(i)
#print(corpus)

In [51]:
test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x:[lemmatizer.lemmatize(i) for i in x])
test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x:' '.join(x))

In [52]:
# Creating Polarity Column using TextBlob
tb_polarity = []
for sentence in train_data['tokenized_text']:
    temp = TextBlob(sentence)
    tb_polarity.append(temp.sentiment[0])
train_data['polarity'] = tb_polarity

test_polarity =[]
for sentence in test_data['tokenized_text']:
    temp = TextBlob(sentence)
    test_polarity.append(temp.sentiment[0])
test_data['polarity'] = test_polarity

In [53]:
train_data['polarity'].isnull().sum()

0

In [54]:
test_data['polarity'].isnull().sum()

0

In [55]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()

tweets_train = tfidf_vectorizer.fit_transform(train_data['tokenized_text'])

In [56]:
tweets_test = tfidf_vectorizer.transform(test_data['tokenized_text'])

In [57]:
tweets_train.shape

(7148, 7795)

In [58]:
X=tweets_train.toarray()
X = pd.DataFrame(X)
print(X.shape, train_data.shape)

X.index = train_data.index

X['polarity'] = train_data['polarity']
#train_data['sentiment']

(7148, 7795) (7148, 6)


In [59]:
test=tweets_test.toarray()
test = pd.DataFrame(test)
print(test.shape, test_data.shape)

test.index = test_data.index

test['polarity'] = test_data['polarity']
#train_data['sentiment']

(1819, 7795) (1819, 5)


In [60]:
X_train,X_val,y_train,y_val = train_test_split(X, train_data.sentiment, test_size=0.2,random_state=22)
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.25,random_state=22)

# Instantiate SMOTE
smote = SMOTE(random_state=9)

# fit_sample onm training data
X_smote,y_smote = smote.fit_sample(X_train,y_train)

# SVC Model

In [61]:
svc_model_smote = SVC(C=1, gamma = 1, kernel = 'rbf')

svc_model_smote.fit(X_smote,y_smote)

pred_test = svc_model_smote.predict(X_test)
pred_val = svc_model_smote.predict(X_val)

score_svc_test  = f1_score(y_test, pred_test, average='weighted')
score_svc_val  = f1_score(y_val, pred_val, average='weighted')

print('SVC Scores for test and val are : \n', score_svc_test, score_svc_val)

SVC Scores for test and val are : 
 0.6687495158968514 0.6864227204915814


# Random Forest Classifier

In [62]:
rf_model_smote = RandomForestClassifier()

rf_model_smote.fit(X_smote,y_smote)

pred_test = rf_model_smote.predict(X_test)
pred_val = rf_model_smote.predict(X_val)

score_rf_test  = f1_score(y_test, pred_test, average='weighted')
score_rf_val  = f1_score(y_val, pred_val, average='weighted')

print('RandomForest Scores for test and val are : \n', score_rf_test, score_rf_val)

RandomForest Scores for test and val are : 
 0.6689489820524303 0.6670407429110558


# LogisticRegression

In [63]:
lr_model_smote = LogisticRegression()

lr_model_smote.fit(X_smote,y_smote)

pred_test = lr_model_smote.predict(X_test)
pred_val = lr_model_smote.predict(X_val)

score_lr_test  = f1_score(y_test, pred_test, average='weighted')
score_lr_val  = f1_score(y_val, pred_val, average='weighted')

print('LogisticRegression Scores for test and val are : \n', score_lr_test, score_lr_val)

LogisticRegression Scores for test and val are : 
 0.6647128956616172 0.6734820512561245


   # Decision Tree

In [64]:
dt_model_smote = DecisionTreeClassifier()

dt_model_smote.fit(X_smote,y_smote)

pred_test = dt_model_smote.predict(X_test)
pred_val = dt_model_smote.predict(X_val)

score_dt_test  = f1_score(y_test, pred_test, average='weighted')
score_dt_val  = f1_score(y_val, pred_val, average='weighted')

print('DecisionTree Scores for test and val are : \n', score_dt_test, score_dt_val)

DecisionTree Scores for test and val are : 
 0.6051563890866247 0.6134949478672181


In [71]:
print('after advanced cleaning with user handles, RT and links:\n')
print('SVC Scores for test and val are : \n', score_svc_test, score_svc_val)
print('-'*100)
print('RandomForest Scores for test and val are : \n', score_rf_test, score_rf_val)
print('-'*100)
print('LogisticRegression Scores for test and val are : \n', score_lr_test, score_lr_val)
print('-'*100)
print('DecisionTree Scores for test and val are : \n', score_dt_test, score_dt_val)

after advanced cleaning with user handles, RT and links:

SVC Scores for test and val are : 
 0.6687495158968514 0.6864227204915814
----------------------------------------------------------------------------------------------------
RandomForest Scores for test and val are : 
 0.6689489820524303 0.6670407429110558
----------------------------------------------------------------------------------------------------
LogisticRegression Scores for test and val are : 
 0.6647128956616172 0.6734820512561245
----------------------------------------------------------------------------------------------------
DecisionTree Scores for test and val are : 
 0.6051563890866247 0.6134949478672181


In [72]:
ID = test_data['tweet_id']
len(ID)

1819

In [ ]:
pred = svc_model_smote.predict(test)

In [68]:
prediction = pd.DataFrame(pred,columns=['sentiment'])
prediction

In [69]:
sub_3 = pd.concat([ID,prediction['sentiment']],1)
sub_3

In [70]:
sub_3.to_csv('sub_3.csv',index=False)